<a href="https://colab.research.google.com/github/yukismd/DL_for_ImageData_and_Finetuning/blob/main/text_classification_hatespeech/PT/bert-large-japanese-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
MODEL = 'tohoku-nlp/bert-large-japanese-v2'

In [2]:
!pwd

/content


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls

drive  sample_data


In [5]:
!ls drive/MyDrive/experiment_results

hs_txt_classification


In [6]:
SAVE_DIR = 'drive/MyDrive/experiment_results/hs_txt_classification'

In [7]:
!pip install datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00


In [8]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [9]:
!pip install fugashi unidic_lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic_lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=ed9d870ce5be54c985a300c66bef385dd73ce406a8b295dd17a70f9ad53d4a52
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic_lite


In [10]:
import datasets
print(datasets.__version__)
import transformers
print(transformers.__version__)
from transformers import AutoTokenizer
import evaluate
print(evaluate.__version__)

import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os
import json

2.19.1
4.41.1
0.4.2


## 学習データ, テストデータ

In [11]:
df = pd.read_csv('https://sample-data-open.s3.ap-northeast-1.amazonaws.com/text_classification_hatespeech/hatespeech_train.csv')
print(df.shape)

(4656, 5)


In [12]:
df_train = df[df['hold'] != 0]    # 学習データ
df_valid = df[df['hold'] == 0]    # 検証データ
df_train.shape, df_valid.shape

((3759, 5), (897, 5))

In [13]:
ds_train = datasets.Dataset.from_pandas(df_train[['text','label']], preserve_index=False)
ds_valid = datasets.Dataset.from_pandas(df_valid[['text','label']], preserve_index=False)

In [14]:
df_test = pd.read_csv('https://sample-data-open.s3.ap-northeast-1.amazonaws.com/text_classification_hatespeech/hatespeech_test.csv')
print(df_test.shape)

(600, 4)


In [15]:
ds_test = datasets.Dataset.from_pandas(df_test[['text','label']], preserve_index=False)

## トークナイザー

In [16]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

BertJapaneseTokenizer(name_or_path='tohoku-nlp/bert-large-japanese-v2', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [17]:
def tokenize_function(examples):
    return tokenizer(examples["text"],
                     truncation=True,
                     max_length=128)    # 学習データの文長が短いので128に設定

In [18]:
ds_tkn_train = ds_train.map(tokenize_function, batched=True)
ds_tkn_valid = ds_valid.map(tokenize_function, batched=True)
ds_tkn_train, ds_tkn_valid

Parameter 'function'=<function tokenize_function at 0x7e295137cdc0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/3759 [00:00<?, ? examples/s]

Map:   0%|          | 0/897 [00:00<?, ? examples/s]

(Dataset({
     features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 3759
 }),
 Dataset({
     features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 897
 }))

In [19]:
# textカラムの削除
ds_tkn_train = ds_tkn_train.remove_columns(['text'])
ds_tkn_valid = ds_tkn_valid.remove_columns(['text'])

# labelカラムをlabelsへリネーム
ds_tkn_train = ds_tkn_train.rename_column('label', 'labels')
ds_tkn_valid = ds_tkn_valid.rename_column('label', 'labels')

In [20]:
ds_tkn_test = ds_test.map(tokenize_function, batched=True)
ds_tkn_test

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 600
})

In [21]:
# textカラムの削除
ds_tkn_test = ds_tkn_test.remove_columns(['text'])

# labelカラムをlabelsへリネーム
ds_tkn_test = ds_tkn_test.rename_column('label', 'labels')

## モデル

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL,
                                                           num_labels=2)

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tohoku-nlp/bert-large-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 学習

In [24]:
metric = evaluate.load("f1")

# 精度評価関数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [26]:
def do_training():
  print('N_EPOCH: ', N_EPOCH)
  print('BATCH_SIZE: ', BATCH_SIZE)
  print('LR', LR)

  trainer_args = TrainingArguments(
    seed=123,
    output_dir=SAVE_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    #save_strategy="epoch",
    save_steps=1e6, # saveのステップを大きくしてここではモデルの保存を行わないようにする
    log_level="critical",
    num_train_epochs=N_EPOCH,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=12,
    save_total_limit=1,
    fp16=True,
    remove_unused_columns=False,
    report_to="none"
    )

  trainer = Trainer(
    model=model,
    args=trainer_args,
    tokenizer=tokenizer,
    train_dataset=ds_tkn_train,
    eval_dataset=ds_tkn_valid,
    compute_metrics=compute_metrics,
    )

  #return trainer

  print('----------  Start Training  ----------')
  trainer.train()

  print('----------  Res on Valid  ----------')
  res_json = trainer.evaluate()
  print(res_json)

  print('----------  Res on Test  ----------')
  test_preds = trainer.predict(ds_tkn_test)
  test_metric = test_preds.metrics
  print(test_metric)

  return res_json, test_metric


In [27]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.000003  # 3e-6

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch16_Lr3e-06
N_EPOCH:  3
BATCH_SIZE:  16
LR 3e-06


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------


Epoch,Training Loss,Validation Loss,F1
1,0.223300,0.133668,0.039216
2,0.129600,0.120316,0.581395
3,0.084000,0.128964,0.640000


----------  Res on Valid  ----------


{'eval_loss': 0.12896382808685303, 'eval_f1': 0.64, 'eval_runtime': 3.3439, 'eval_samples_per_second': 268.248, 'eval_steps_per_second': 22.429, 'epoch': 3.0}
----------  Res on Test  ----------
{'test_loss': 0.1441727727651596, 'test_f1': 0.591549295774648, 'test_runtime': 2.1907, 'test_samples_per_second': 273.88, 'test_steps_per_second': 22.823}


In [28]:
# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

drive/MyDrive/experiment_results/hs_txt_classification/VALID__bert-large-japanese-v2_Epoch3_Batch16_Lr3e-06.json
drive/MyDrive/experiment_results/hs_txt_classification/TEST__bert-large-japanese-v2_Epoch3_Batch16_Lr3e-06.json


In [30]:
!ls drive/MyDrive/experiment_results/hs_txt_classification

checkpoint-940	TEST__bert-large-japanese-v2_Epoch3_Batch16_Lr3e-06.json
OLD		VALID__bert-large-japanese-v2_Epoch3_Batch16_Lr3e-06.json


In [31]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00001  # 1e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch16_Lr1e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 1e-05
----------  Start Training  ----------


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'loss': 0.1003, 'grad_norm': 0.34371742606163025, 'learning_rate': 6.680851063829787e-06, 'epoch': 1.0}
{'eval_loss': 0.23666493594646454, 'eval_f1': 0.582089552238806, 'eval_runtime': 3.2997, 'eval_samples_per_second': 271.839, 'eval_steps_per_second': 22.729, 'epoch': 1.0}
{'loss': 0.0237, 'grad_norm': 0.2034163922071457, 'learning_rate': 3.3617021276595745e-06, 'epoch': 2.0}
{'eval_loss': 0.23756766319274902, 'eval_f1': 0.6250000000000001, 'eval_runtime': 3.1277, 'eval_samples_per_second': 286.797, 'eval_steps_per_second': 23.98, 'epoch': 2.0}
{'loss': 0.006, 'grad_norm': 0.002590178744867444, 'learning_rate': 2.836879432624114e-08, 'epoch': 3.0}
{'eval_loss': 0.2829039990901947, 'eval_f1': 0.6296296296296295, 'eval_runtime': 3.1998, 'eval_samples_per_second': 280.332, 'eval_steps_per_second': 23.439, 'epoch': 3.0}
{'train_runtime': 231.9468, 'train_samples_per_second': 48.619, 'train_steps_per_second': 3.039, 'train_loss': 0.043321255927390244, 'epoch': 3.0}
----------  Res on Val

In [32]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00003  # 3e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch16_Lr3e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 3e-05
----------  Start Training  ----------


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'loss': 0.0745, 'grad_norm': 0.017445947974920273, 'learning_rate': 2.0127659574468084e-05, 'epoch': 1.0}
{'eval_loss': 0.2768104076385498, 'eval_f1': 0.576271186440678, 'eval_runtime': 3.2965, 'eval_samples_per_second': 272.105, 'eval_steps_per_second': 22.751, 'epoch': 1.0}
{'loss': 0.0415, 'grad_norm': 0.4697950482368469, 'learning_rate': 1.0127659574468085e-05, 'epoch': 2.0}
{'eval_loss': 0.23500892519950867, 'eval_f1': 0.6355140186915889, 'eval_runtime': 3.631, 'eval_samples_per_second': 247.04, 'eval_steps_per_second': 20.656, 'epoch': 2.0}
{'loss': 0.0079, 'grad_norm': 0.004685573745518923, 'learning_rate': 1.276595744680851e-07, 'epoch': 3.0}
{'eval_loss': 0.2110576331615448, 'eval_f1': 0.7096774193548386, 'eval_runtime': 3.188, 'eval_samples_per_second': 281.367, 'eval_steps_per_second': 23.526, 'epoch': 3.0}
{'train_runtime': 233.8102, 'train_samples_per_second': 48.231, 'train_steps_per_second': 3.015, 'train_loss': 0.04130297484972798, 'epoch': 3.0}
----------  Res on Vali

In [33]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00005  # 5e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch16_Lr5e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 5e-05
----------  Start Training  ----------


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'loss': 0.1142, 'grad_norm': 0.36031991243362427, 'learning_rate': 3.361702127659575e-05, 'epoch': 1.0}
{'eval_loss': 0.17552554607391357, 'eval_f1': 0.5964912280701754, 'eval_runtime': 3.1114, 'eval_samples_per_second': 288.294, 'eval_steps_per_second': 24.105, 'epoch': 1.0}
{'loss': 0.0554, 'grad_norm': 0.08420763164758682, 'learning_rate': 1.695035460992908e-05, 'epoch': 2.0}
{'eval_loss': 0.2975218892097473, 'eval_f1': 0.38095238095238093, 'eval_runtime': 3.2014, 'eval_samples_per_second': 280.187, 'eval_steps_per_second': 23.427, 'epoch': 2.0}
{'loss': 0.0193, 'grad_norm': 0.03323858603835106, 'learning_rate': 2.8368794326241136e-07, 'epoch': 3.0}
{'eval_loss': 0.2328931838274002, 'eval_f1': 0.5957446808510639, 'eval_runtime': 3.1966, 'eval_samples_per_second': 280.611, 'eval_steps_per_second': 23.462, 'epoch': 3.0}
{'train_runtime': 229.9066, 'train_samples_per_second': 49.05, 'train_steps_per_second': 3.066, 'train_loss': 0.0629548282488018, 'epoch': 3.0}
----------  Res on Val

In [ ]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.000003  # 3e-6

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)